In [18]:
import os
import tensorflow as tf
import time
import csv

In [2]:
def decode(serialized_example):
    alphabets=list()
    features = tf.parse_single_example(
      serialized_example,      
      features={
          'label': tf.FixedLenFeature([], tf.int64),
          'cho_label':tf.FixedLenFeature([], tf.int64),
          'jung_label':tf.FixedLenFeature([], tf.int64),
          'jong_label':tf.FixedLenFeature([], tf.int64),
          'width': tf.FixedLenFeature([], tf.int64),
          'height': tf.FixedLenFeature([], tf.int64),
          'image': tf.FixedLenFeature([], tf.string)
      })
    
    image = tf.decode_raw(features['image'],tf.uint8)
    #image = tf.reshape(image,[64,64,1])
    image = tf.cast(image, tf.float32)
    image = tf.reshape(image, [64,64,1])
    label = tf.cast(features['label'],tf.int64)
    cho_label=tf.cast(features['cho_label'],tf.int64)
    jung_label=tf.cast(features['jung_label'],tf.int64)
    jong_label=tf.cast(features['jong_label'],tf.int64)
    
    return image,label, cho_label, jung_label, jong_label

In [3]:
batch_size=100

train_file_name = "Shuffled_with_alphabet_PE92_Train.tfrecord"
train_dataset = tf.data.TFRecordDataset("./"+train_file_name)
train_dataset = train_dataset.map(decode)
train_dataset = train_dataset.repeat()
train_dataset= train_dataset.shuffle(buffer_size=10000)
train_dataset = train_dataset.batch(batch_size)

In [4]:
test_file_name = "Shuffled_with_alphabet_PE92_Test.tfrecord"
test_dataset = tf.data.TFRecordDataset("./"+test_file_name)
test_dataset = test_dataset.map(decode)
test_dataset = test_dataset.repeat()
test_dataset= test_dataset.shuffle(buffer_size=10000)
test_dataset = test_dataset.batch(batch_size)

In [5]:
train_data_length = sum(1 for _ in tf.python_io.tf_record_iterator("./"+train_file_name))

test_data_length = sum(1 for _ in tf.python_io.tf_record_iterator("./"+test_file_name))

print(train_data_length)

200215


In [6]:
train_iterator = tf.data.Iterator.from_structure(train_dataset.output_types,
                                           train_dataset.output_shapes)
test_iterator = tf.data.Iterator.from_structure(test_dataset.output_types,
                                           test_dataset.output_shapes)

train_init_op = train_iterator.make_initializer(train_dataset)
test_init_op = test_iterator.make_initializer(test_dataset)

train_image, train_label, train_cho, train_jung, train_jong = train_iterator.get_next()
test_image, test_label, test_cho, test_jung, test_jong = test_iterator.get_next()

train_total_batch = int(train_data_length/batch_size)
test_total_batch = int(test_data_length/batch_size)

In [7]:
input_images = tf.placeholder(tf.float32,[None,64,64,1])
input_label_cho = tf.placeholder(tf.int64,[None])
input_label_jung = tf.placeholder(tf.int64,[None])
input_label_jong = tf.placeholder(tf.int64,[None])

is_training = tf.placeholder(tf.bool)
keep_prob = tf.placeholder(tf.float32)
print(input_images)


Tensor("Placeholder:0", shape=(?, 64, 64, 1), dtype=float32)


In [8]:
W1 = tf.Variable(tf.random_normal([5,5,1,32], stddev=0.01))
L1 = tf.nn.conv2d(input_images,W1, strides=[1,1,1,1], padding = 'SAME')
print(L1)
bn1 = tf.layers.batch_normalization(L1, training=is_training)
at1 = tf.nn.relu(bn1)

pool1 =  tf.nn.max_pool(at1, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
print(pool1)

Tensor("Conv2D:0", shape=(?, 64, 64, 32), dtype=float32)
Tensor("MaxPool:0", shape=(?, 32, 32, 32), dtype=float32)


In [9]:
W2 = tf.Variable(tf.random_normal([5, 5, 32, 64], stddev=0.01))

L2 = tf.nn.conv2d(pool1,W2, strides=[1,1,1,1], padding = 'SAME')
bn2 = tf.layers.batch_normalization(L2, training=is_training)
at2 = tf.nn.relu(bn2)

pool2 =  tf.nn.max_pool(at2, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
print(pool2)

Tensor("MaxPool_1:0", shape=(?, 16, 16, 64), dtype=float32)


In [10]:
W3 = tf.Variable(tf.random_normal([4, 4, 64, 128], stddev=0.01))

L3 = tf.nn.conv2d(pool2,W3, strides=[1,1,1,1], padding = 'SAME')
bn3 = tf.layers.batch_normalization(L3, training=is_training)
at3 = tf.nn.relu(bn3)

pool3 =  tf.nn.max_pool(at3, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
print(pool3)

Tensor("MaxPool_2:0", shape=(?, 8, 8, 128), dtype=float32)


In [11]:
W4 = tf.Variable(tf.random_normal([4, 4, 128, 256], stddev=0.01))

L4 = tf.nn.conv2d(pool3,W4, strides=[1,1,1,1], padding = 'SAME')
bn4 = tf.layers.batch_normalization(L4, training=is_training)
at4 = tf.nn.relu(bn4)

pool4 =  tf.nn.max_pool(at4, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
print(pool4)

Tensor("MaxPool_3:0", shape=(?, 4, 4, 256), dtype=float32)


In [12]:
W_cho1 = tf.Variable(tf.random_normal([4*4*256, 128], stddev=0.01))
W_jung1 = tf.Variable(tf.random_normal([4*4*256, 128], stddev=0.01))
W_jong1 = tf.Variable(tf.random_normal([4*4*256, 128], stddev=0.01))
fc = tf.reshape(pool4,[-1,4*4*256])

fc_cho = tf.matmul(fc, W_cho1)
fc_cho = tf.nn.relu(fc_cho)

fc_jung = tf.matmul(fc, W_jung1)
fc_jung = tf.nn.relu(fc_jung)

fc_jong = tf.matmul(fc, W_jong1)
fc_jong = tf.nn.relu(fc_jong)


fc_cho= tf.nn.dropout(fc_cho,keep_prob)
fc_jung= tf.nn.dropout(fc_jung,keep_prob)
fc_jong= tf.nn.dropout(fc_jong,keep_prob)

print(fc)

Tensor("Reshape:0", shape=(?, 4096), dtype=float32)


In [13]:
W_cho2 = tf.Variable(tf.random_normal([128,19],stddev=0.01))
W_jung2 = tf.Variable(tf.random_normal([128,21],stddev=0.01))
W_jong2 = tf.Variable(tf.random_normal([128,28],stddev=0.01))

logit_cho = tf.matmul(fc_cho, W_cho2)
logit_jung = tf.matmul(fc_jung, W_jung2)
logit_jong = tf.matmul(fc_jong, W_jong2)

print(logit_cho)

Tensor("MatMul_3:0", shape=(?, 19), dtype=float32)


In [14]:
cross_entropy_cho = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logit_cho, labels=input_label_cho)
cross_entropy_jung = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logit_jung, labels=input_label_jung)
cross_entropy_jong = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logit_jong, labels=input_label_jong)

loss_cho = tf.reduce_mean(cross_entropy_cho)
loss_jung = tf.reduce_mean(cross_entropy_jung)
loss_jong = tf.reduce_mean(cross_entropy_jong)

total_loss =tf.reduce_mean(tf.stack([loss_cho, loss_jung, loss_jong]))

update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    optimizer = tf.train.AdamOptimizer(0.001).minimize(total_loss)
loss_summ=tf.summary.scalar("Loss",total_loss)
print(total_loss)

Tensor("Mean_3:0", shape=(), dtype=float32)


In [15]:
argmax_cho = tf.argmax(logit_cho,1)
argmax_jung = tf.argmax(logit_jung,1)
argmax_jong = tf.argmax(logit_jong,1)
print(argmax_cho)
prediction = tf.stack([argmax_cho,argmax_jung,argmax_jong],1)
print(total_class)

Tensor("ArgMax:0", shape=(?,), dtype=int64)
Tensor("stack_1:0", shape=(?, 3), dtype=int64)


In [16]:
total_labels = tf.stack([input_label_cho, input_label_jung, input_label_jong],1)
equal_check = tf.equal(prediction,total_labels)
cast_equal = tf.cast(equal_check, tf.float32)
reduce_min = tf.reduce_min(cast_equal,1)

accuracy = tf.reduce_mean(reduce_min)
acc_summ=tf.summary.scalar("Accuracy",accuracy)

In [19]:
init = tf.group(tf.global_variables_initializer(),tf.local_variables_initializer())
saver = tf.train.Saver()

print("Train total Batch :%d" % (train_total_batch))
with tf.Session() as sess:
    
    predict_file = open('./save/PE92/CNNV2/prediction.csv','w',encoding='utf-8',newline='')
    label_file = open('./save/PE92/CNNV2/labels.csv','w',encoding='utf-8',newline='')
    pred_writer = csv.writer(predict_file, delimiter=',')
    label_writer = csv.writer(label_file, delimiter=',')
    
    merged_summary = tf.summary.merge_all()
    train_writer = tf.summary.FileWriter("./logs/CNNV2/PE92/Train")
    test_writer = tf.summary.FileWriter("./logs/CNNV2/PE92/Test")
    
    sess.run(init)
    coord=tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    sess.run(train_init_op)
    sess.run(test_init_op)
    #Train Model
    
    start_time = time.time()
    for epoch in range(50):
        for i in range(train_total_batch):
        #for i in range(batch_size):
            batch_images, batch_cho, batch_jung, batch_jong = sess.run([train_image, train_cho, train_jung, train_jong])
            
            sess.run(optimizer,feed_dict={input_images: batch_images, input_label_cho:batch_cho, 
                                                               input_label_jung : batch_jung, input_label_jong : batch_jong, keep_prob:0.5, is_training:True})
            summary, train_acc, train_loss = sess.run([merged_summary, accuracy,total_loss], feed_dict={input_images: batch_images, input_label_cho:batch_cho, 
                                                               input_label_jung : batch_jung, input_label_jong : batch_jong, keep_prob:1.0, is_training:False})
            train_writer.add_summary(summary,i)
            if(i%100==0):
                print('Epoch :%d | step %d, training accuracy: %f, train loss: %f' % (epoch,i, train_acc,train_loss))

        save_path = saver.save(sess, "save/PE92/CNNV2/model_PE92.ckpt")
       
        batch_images, batch_cho, batch_jung, batch_jong = sess.run([test_image, test_cho, test_jung, test_jong])
        
        summary, test_acc, test_loss, test_predicton,labels=sess.run([merged_summary, accuracy,total_loss,prediction, total_labels], feed_dict={input_images: batch_images, input_label_cho:batch_cho, 
                                                               input_label_jung : batch_jung, input_label_jong : batch_jong, keep_prob:0.5, is_training:False})
        test_writer.add_summary(summary,epoch)
        pred_writer.writerow(test_prediction)
        label_writer.writerow(labels)
        print("Test Acc : %f, Test Loss :%f" %(test_acc,test_loss))
            
        end_time = time.time()
        elapsed = end_time - start_time
        elapsed = elapsed/60
        print("Time : %f minutes" %(elapsed))
    
    print("Training is done.")
    
    #Test Model
    coord.request_stop()
    coord.join(threads)
    predict_file.close()
    label_file.close()
    train_writer.close()
    test_writer.close()

Train total Batch :2002
Epoch :0 | step 0, training accuracy: 0.000000, train loss: 3.063150
Epoch :0 | step 100, training accuracy: 0.020000, train loss: 1.989571
Epoch :0 | step 200, training accuracy: 0.120000, train loss: 1.390005
Epoch :0 | step 300, training accuracy: 0.290000, train loss: 0.935469
Epoch :0 | step 400, training accuracy: 0.220000, train loss: 1.148008
Epoch :0 | step 500, training accuracy: 0.310000, train loss: 0.810537
Epoch :0 | step 600, training accuracy: 0.600000, train loss: 0.480790
Epoch :0 | step 700, training accuracy: 0.540000, train loss: 0.565651
Epoch :0 | step 800, training accuracy: 0.680000, train loss: 0.372070
Epoch :0 | step 900, training accuracy: 0.540000, train loss: 0.604798
Epoch :0 | step 1000, training accuracy: 0.610000, train loss: 0.510342
Epoch :0 | step 1100, training accuracy: 0.660000, train loss: 0.377231
Epoch :0 | step 1200, training accuracy: 0.740000, train loss: 0.349815
Epoch :0 | step 1300, training accuracy: 0.710000, t

NameError: name 'test_prediction' is not defined

In [ ]:
train_file_name = "Shuffled_with_alphabet_HanDB_Train.tfrecord"
train_dataset = tf.data.TFRecordDataset("./"+train_file_name)
train_dataset = train_dataset.map(decode)
train_dataset = train_dataset.repeat()
train_dataset= train_dataset.shuffle(buffer_size=10000)
train_dataset = train_dataset.batch(batch_size)

test_file_name = "Shuffled_with_alphabet_HanDB_Test.tfrecord"
test_dataset = tf.data.TFRecordDataset("./"+test_file_name)
test_dataset = test_dataset.map(decode)
test_dataset = test_dataset.repeat()
test_dataset= test_dataset.shuffle(buffer_size=10000)
test_dataset = test_dataset.batch(batch_size)

train_data_length = sum(1 for _ in tf.python_io.tf_record_iterator("./"+train_file_name))
print(train_data_length)

test_data_length = sum(1 for _ in tf.python_io.tf_record_iterator("./"+test_file_name))


train_iterator = tf.data.Iterator.from_structure(train_dataset.output_types,
                                           train_dataset.output_shapes)
test_iterator = tf.data.Iterator.from_structure(test_dataset.output_types,
                                           test_dataset.output_shapes)

train_init_op = train_iterator.make_initializer(train_dataset)
test_init_op = test_iterator.make_initializer(test_dataset)

train_image, train_label, train_cho, train_jung, train_jong = train_iterator.get_next()
test_image, test_label, test_cho, test_jung, test_jong = test_iterator.get_next()

train_total_batch = int(train_data_length/batch_size)
test_total_batch = int(test_data_length/batch_size)

In [ ]:
init = tf.group(tf.global_variables_initializer(),tf.local_variables_initializer())
saver = tf.train.Saver()

print("Train total Batch :%d" % (train_total_batch))
with tf.Session() as sess:
    
        predict_file = open('./save/HanDB/CNNV2/prediction.csv','w',encoding='utf-8',newline='')
    label_file = open('./save/HanDB/CNNV2/labels.csv','w',encoding='utf-8',newline='')
    pred_writer = csv.writer(predict_file, delimiter=',')
    label_writer = csv.writer(label_file, delimiter=',')
    
    merged_summary = tf.summary.merge_all()
    train_writer = tf.summary.FileWriter("./logs/CNNV2/HanDB/Train")
    test_writer = tf.summary.FileWriter("./logs/CNNV2/HanDB/Test")
    
    sess.run(init)
    coord=tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    sess.run(train_init_op)
    sess.run(test_init_op)
    #Train Model
    
    start_time = time.time()
    for epoch in range(50):
        for i in range(train_total_batch):
        #for i in range(batch_size):
            batch_images, batch_cho, batch_jung, batch_jong = sess.run([train_image, train_cho, train_jung, train_jong])
            
            sess.run(optimizer,feed_dict={input_images: batch_images, input_label_cho:batch_cho, 
                                                               input_label_jung : batch_jung, input_label_jong : batch_jong, keep_prob:0.5, is_training:True})
            summary, train_acc, train_loss = sess.run([merged_summary, accuracy,total_loss], feed_dict={input_images: batch_images, input_label_cho:batch_cho, 
                                                               input_label_jung : batch_jung, input_label_jong : batch_jong, keep_prob:1.0, is_training:False})
            train_writer.add_summary(summary,i)
            if(i%100==0):
                print('Epoch :%d | step %d, training accuracy: %f, train loss: %f' % (epoch,i, train_acc,train_loss))

        save_path = saver.save(sess, "save/HanDB/CNNV2/model_HanDB.ckpt")
       
        batch_images, batch_cho, batch_jung, batch_jong = sess.run([test_image, test_cho, test_jung, test_jong])
        
        summary, test_acc, test_loss, test_predicton,labels = sess.run([merged_summary, accuracy,total_loss,prediction, total_labels], feed_dict={input_images: batch_images, input_label_cho:batch_cho, 
                                                               input_label_jung : batch_jung, input_label_jong : batch_jong, keep_prob:0.5, is_training:False})
        test_writer.add_summary(summary,epoch)
        pred_writer.writerow(test_prediction)
        label_writer.writerow(labels)

        print("Test Acc : %f, Test Loss :%f" %(test_acc,test_loss))
            
        end_time = time.time()
        elapsed = end_time - start_time
        elapsed = elapsed/60
        print("Time : %f minutes" %(elapsed))
    
    print("Training is done.")
    
    #Test Model
    coord.request_stop()
    coord.join(threads)
    predict_file.close()
    label_file.close()
    train_writer.close()
    test_writer.close()

In [19]:
with tf.Session() as sess:
    sess.run(init)
    coord=tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    
    sess.run(train_init_op)
    start_time = time.time()
    for i in range(train_total_batch):
        #for i in range(batch_size):
        batch_images, batch_labels = sess.run([train_image, train_label])
        print(batch_labels)

        
    coord.request_stop()
    coord.join(threads)

[1085 1829 1497 1596 2036  364 1661 1894 1109  129  610 2000 1486   71
 1599 2103 1562 1413 1683 1879 1177  692  971 1782  759 1454  450  754
 1660 1103 1748 2224 1596   51  987 1172 2053   51 1414  832 1720  901
  229 2155 2228 2247  908  987   91 1510 1839 2007 1171 1155   60 1828
 1851  243 1324 1787  709 1405  942  432  590 2270 2315 1419  889  165
  602 1555 1429 1619  590 2346 1469 1209 1454  163  336  966  681 2285
  698  955 1031  275 1452 1979 2346 1807  920 2213  149 1809 1386 2225
  336   18]
[1949 2228 1529 1598 2116 2343  647 1593  824  691 2032 1214 2053 1365
 1920  163  244  458  364 2277 1555 1873 2171  366 1217 1418  406 1532
  908 1554  964 2213   58  330   60 1419  349  697  647 2140  517 2279
 1689 2217 1848 1419 1535 1676 1402  907 1692  705  909 1462 1654 1962
 1806   31 1426 1494  649  460  620 2107 1873  446  540 1572  650 1683
  900  909 2225 1034 1345  936 1536 1594  396  329  117  647  500 2336
 1419  904  515 1363 1000  613  954  802  835  542 1624  163  769

[ 784  902 2028  325 1398 1708 1851 1886 1397  773 1356   91  905  267
   56 1839  359 2007 1494 2048 1355 1529  424 1412  547  384  791 1574
  446 1944 1683 1441  648  740 1689  560  832 1354  205 1821  122 1255
   69 1279  691  407 1894 2180   18  494 1984  651 1393  728 1345 1855
  746 1245  728  835 1598 1890  723  349  772   91 2198  340 2103  508
  582 1789  989 1889 1484 1430 1260 2312   91 1418 2279  244  959 1802
 1554  300  856 2041  509 1999 2228 1873   62  359  409  406 1378  494
  769  641]
[2058 1530 1155 1394 2260 1652 2228   26  889  788  856 1821 1563 1511
 1256   40  399 1258   15  119  963 1827  893 1782  570 1533 2269 1328
 1266 1889  986 1487 1685 1355 2006  677 1554 1366 1186    0 1126  173
  736  873 1263 1675  832 1150  460  349 1495  766   89  703  183 1682
 1986 2233 1267 2171 1652  891  153 1258 1430 1892  112 2180  410 2126
 1180 2211  889 1487 1984 1405  760  581 1810 1838  461  985  375  536
 2268 1349   91 1886 1542 1725   57  955  917  148  474  964 1172

[1345 2177 1518  931 1572 2156 1519  118 1799  704 1386 2054 1357 1141
  329  581  112 1938 1532  619  384 2321  861 1682  672  893  433   89
  955  589 1491  907 1653  361  296 1805  728  768 2342 1031 1765 1980
 1107 1356 1630 2091 1530  515  729  302  155  345 2024  492 2268 1765
  878 1817 1031 2239  539  677  282 1890  547 1683  942  964 1788 1441
 1037  935 1680  527  602 1924  112 2343    0  884 1213  705 1828 2071
  463 1346  906  937 1429   69 2051 1401 1881  384 1374  959  138 1298
  971  985]
[2081   12  386 1495  769 1144 2081 1237   59 1518  581 1395    0  175
  579 1452 1707 2258 1660 2058 1561  358 1258 2174 1116 1153 2048 2081
 2209 1690  828 1839 1631  813  676 2144   76  908  822 2115 1177  364
 1651 1621   91 1989 1619  330 1107   91 1393 1954  152  746   40   60
 1773  681 1984   11 1518  345 1787 1360 2007 1446 1112  788 2228 1929
  570  481 2177 1547  915 1363  579  960 2222 1003 1395 1533  560 2166
  183 1181  170 2054 1972  122  758  931 1031 1214 1690  560 2268

[1487  283  901 1787  170 1322 1485 1022  531   89  121 1550  851  163
 1241 1452  884  773  308 1438 1780 1535  542  128 1654  422 1398  733
  299  577   64 1566  824 2066  291   13 1429 1804 1829 2233  531 1810
  560 1982 2007 1425  682 2225 2154 1555   94  757 1627 1705 1857    0
 1262  386 1177  803 1214  832  117 2102  544 2182 2214 2286  183 2064
  915  705    7  903  549  255 1989  403 1549  681 1266  597  407 2250
  165 1425  349  349 2228 2293  214 1475 1521 1345 1250 2095 1338 1451
 1210 1518]
[ 579  346 1621 1782  760  751 1433  959 1462 2247 2227 2333 2028  677
  712 1404  446 1107 1435  646   22  917  490  428  915 1532 1116 1790
 1748 1215 1488 1255  786  869 1378  681 1742 1124 2125  648  630 1991
  647 2200  774  656 1117  517  358  907  776   60 1881 2125 1733  906
 1813  954   80 1084   45  703  769 1284 1461 1318 1263  648 1699  161
 1387   29  954  891 1107  175 1660  561  490  312 2126  205  426  220
  802 1452  829 1430 1922  697  824  908  366 2161  748 1685 1821

[1593 2171   44 1084  715  692 1689 1333 1910  709  546  421 1542 1549
  770  120 1395 1214 1160  531  959 1986 2058  676 2053  758  648 2174
  692 1563  155 2296 2346 1533 1704 1467  391 1572 1596  788 1107  722
 2180 1369   18  140  581 1430  112 1611  451 1550   12 1358 1535 2082
 1356 1880 1031 1357  595 1300 1379  203 1495 1215  758 1979  803 2266
 1827 2272  384  658 2053 1676 2217  579   97 1550  517 1141 1182  148
  651 1104  148 2253 2247  662   57 1685  964 1013 1758  416 1079  159
 1098 1497]
[ 244 1842 2210 1596 2270  975  299 1829 1231  750  942  530  113  907
 2246  837  407 1542   59  937 1363 1462 1839 1894 1213  247 1084   91
  801 1020   64 2016 2217 1037 1557 2273 2008 1370 1653 2149  674  677
  579 1783 1596  854 1920 1335 1685 1031  778 2041  770  802 1357 1652
 1454   69 1624  306  460 1533 1133  927 1394  538 1619 1180  658 1810
 2144  672  681 2209  796 1177 1603 1428   57 1737  704 1828 1689 1966
  432 1620  893 2208  460  580 1200 1180 1503  114   29 2166 1671

...

[ 460  582  414 2293   64  938 1828 1616 1051 1370 1473 1247   57  987
 1884  121  955 1214 1356  636 1673  601  989  461  159 1692  427  931
  145  195 1146 2346  364 1020 1687  384 1018 1566 2255  376  952 1848
  119  728 2273 1492 1685 1807  985  112  904 1554  171 1144  542   37
 1694 2258 1616  698 2233  443 1549  933  297  915 1286  753   19 1910
 1804 2200 1920 1488 1178 2176  128 2259 2227 2154  171 1462 1549 1986
 1596  739 1459  804  964  731  646 1413 1596  265 1078 2259 2241  527
 2259  518]
[ 401 1491 1403 1140  149 1449  407    2 1155 1652 1316 1260 1249  672
  537  674  692 2115 1550  364  145  515 1485 2242 1601 1403 1744 2259
  767 1168  508  864  310  128  294  828 2334 1873    2  204 1855 1635
  791 1413  671 1652 1403 2214 1251 1987 1627  939 1452 1879 1144  831
  437 1782  121  762  757 1593   69 2291 1676  901 1469 1405 1886  329
 1971 2224 1452  155 1467  460  770  369  933  283 1782  202 2148 1733
  908 1809 1614 1805 1886  813    4 2194  908 2339 2235   64 1200

[1365  542 1386 2018  149 1668 1562 1733 1937 1611  553   86 2107  283
  474 2208 2175  161 2240  464  729  121 1680 1928 2293 1963 1805 1928
 1109 1684 1827 2216 1890 1873  549 1485  778  432  929 1691 1273  776
  744 1595 1109 2130 1260 2155 1262 1816  155 2250  765 2209 1603  492
  386  356 1421 1443  761 1461 2058 1214 1566  900  150  676 2194  246
  568 1403 1485 1660 1484 1675 1030  756 1119 1986  917  933  322 1855
 1989 1595 1832 1533    2 1313   40  581  541  104 2150  828 1894  985
  907  294]
[  59  590  261 1692 1260 1738   57  991 1630 2279 1860 1689 2053 2028
 1782 1425 1839 1557  336 1469 2273  450  149  674 2258 1351 1792 2211
 2336 2334 2293 1292 1757 1323 1596 1813 1119  119  238 2150  933  693
 1853 1593  424 2126 1178 2000 1535 1495 1690  729  756  815 1484 1412
 2061 2197 1782 1098  310  155 1910  511  772 2194    4 1527 1744  291
 2150 1358 1485 1828   62  163  299 1266 1345 1205  771 1677 1452 1660
 2007 1928   89  903  964 1345  359  620 2260 1807 2245  674 1554

[1600  151  474  442  163 1199 1413 1107 1566 1405 1402 1689   64 2041
  677  646  669 1555  349  145 1995    1 1116  745 1345 2171 1386  463
  832 1386  813  802  108 1687 1155  345 1267  356 2150  935 1075 1255
  719 1381  416  778 1582    2 1418 1719 1099 2107 1780  797 1430 2015
 1655 1542 2247 1593   62 1880  693 1362  677  931 1353  347  294 1286
  906 2166  170 1860  563  382  283 1595 1598 1146  778  767 2080 1542
  985  901  446 2150  650 1463 2151  778 2150  282  128 1463 2007 2269
  542  258]
[1153  447 1827 1733 1266  796  380 1860 1186 1365 1982  677 1374 1316
 1561  443 1690  509 1403 1107 1547  492  145  539 1155  796 1804 1022
  839  709 2267 1178  788  697  722  769 1873 1599 2095  975  527   44
  171 1594 1365 1100  768  517 1663  625 1790   79 2342  646  699 1487
 1843 1660 1366  559  847 1356 1542 2171 1040 1410 1683 2150 2288  490
  159 2186 1286  778 1441  170  152  958 1654 1984 1894 1030 2106  517
   18  692 1684  784  283 2095  781 2116 2271 1030 1676 1030 1084

[1660 2260  641 2259  914 2000  693  887  207 2224 1624    1  903 1691
 1778 2286  475  465 1119 1881 1600 1793 1495 1141  163   62  884 1263
 1485  904 1737  622 1178 2271 2085  985  128 2233 2266  275 1336 2247
 1266  963 1881 1034 1549 2081  443  703  811  895 1621  282  879 2233
   51 2227   47 2346  118 1533  386  117  658 2154  633  906 2107 1266
 1657 2293 1989  514 1434  788 1304 1910  901  296 2177 1196  461  116
 1077 1463  754 1491 1494  647 2209 1572  540  893 2177 1463  412 2209
  859  822]
[ 698 2149  915 1214 1328 1030 1550  773 2209  760  386 1846  329 1443
  746 1445 1112 2018 1104 1255  605 1804 1214 2324  148 1353  729  371
  637 2280 1535 2028 2194  291  760 1421  668 2007 1247  488 1286 1177
  901  653 1569 1454 1554 1999 1365  443 2233 1722 2252 1495  519 1454
 1180 1574 1494  704 1253 1354 2302 1889 2193 2318  403  735  835  300
 1852 1031 1529 1398 1356 1894  150 2214 1408  302 1178 1864  165   90
  954  460 1787   57 1155 1217 2214 1788 1821  648 1249 1393  359

[2177  884 2007  145  985  784  427  749 1692  517 2224  785 1783  492
 1459 1452 1997 1572 1510  407  908 1984 1886  565 1478  784  599 1603
 1016 1621  668 1421  515 1982 1180  579  245  329 1880 1199 2268  617
 1910  722  933   11  908  835 1486 1418  677   51 1928  437 2237 1555
 1561 1997  294  681 1986  998 1175 1809  154 1013 2070 1800 1461 1652
 1186 1412 1368 1344 1585  756 1651  410  899   84 1652  904  330 1601
 1864   76 1514 1787 1104 1652 1909 2259  366 2053 2253 2230   42   77
 2008  432]
[  97  704  464 1402 2209 2336 1140 2106 2008 2268 2106   18 1412 1821
 1982 1812 1461   18 2229    7 1112 1554  112   62 1827 2065 1138 1430
  835  802 2134 1171  436   91 1563  347  646 1689 1030  803  938 2286
  768 1793 1966 1197  345 1197  692 1787   38 1530 1178 1910 2203 1989
 1780 1345 1881 1534   38 2291 1345 2344 1469 1883 1366  162  767 1144
 1778  145 1690 1498 1848 1171   42 1452  682 1948  801 1905 1218 1418
  128   97  223  132 1873 1363  570 1827 2041  788  488 1924 1260

In [1]:
import tensorflow as tf
from random import randint

dims = 8
pos  = randint(0, dims - 1)
print(pos)
logits = tf.random_uniform([dims], maxval=3, dtype=tf.float32)
labels = tf.one_hot(pos, dims)
print(logits)
print(labels)

res1 = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=labels)
res2 = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=tf.constant(pos))
loss=tf.reduce_mean(res2)
with tf.Session() as sess:
    a, b = sess.run([res1, loss])
    print (a, b)
    print (a == b)

4
Tensor("random_uniform:0", shape=(8,), dtype=float32)
Tensor("one_hot:0", shape=(8,), dtype=float32)
1.1221625 1.1221625
True


In [ ]:
def read_and_decode(filename_queue, batch_size):
    tf_record_reader=tf.TFRecordReader()
    _, tf_record_serialized = tf_record_reader.read(tf_record_filename_queue)

    tf_record_features = tf.parse_single_example(
        tf_record_serialized,
        features={
            'label': tf.FixedLenFeature([], tf.int64),
            'width': tf.FixedLenFeature([], tf.int64),
            'height': tf.FixedLenFeature([], tf.int64),
            'image': tf.FixedLenFeature([], tf.string),
        })
    tf_record_image = tf.decode_raw(tf_record_features['image'],tf.uint8)
    
    tf_record_image = tf.cast(tf_record_image, tf.float32)

    tf_record_image = tf.reshape(tf_record_image, [64,64,1])

    tf_record_label = tf.cast(tf_record_features['label'],tf.int64)
    
    images, labels = tf.train.shuffle_batch([tf_record_image, tf_record_label],
                                         batch_size = batch_size,
                                         capacity=20*batch_size,
                                         min_after_dequeue=5*batch_size )
    return images, labels